## ONNX + TenorRT

There are several pathways one can choose from, in order to take a model from development to production. A clear winner amongst all existing methods is TensorRT, NVIDIA's flagship model optimization framework and inference engine generator. In simple words, TensorRT takes a Torch/TensorFlow model and converts it into an "engine" such that it makes most of the existing hardware resource. We will first look at a simple example of converting a classification model into TensorRT engine. 

In [1]:
from torchvision import models
import cv2
import torch
from torchvision.transforms import Resize, Compose, ToTensor, Normalize
from model.regseg import RegSeg
import onnx

In [2]:
WEIGHTS_PATH = "/home/zeeshan-motorai/workspace/github_repos/motor-ai-organization/tensorrt_torchscript_demo/notebook_regseg/weights/regseg_best_110423"
# ENGINE_PATH = "../trt_engines/regseg_final.engine"
ONNX_FILE_PATH = "/home/zeeshan-motorai/workspace/github_repos/motor-ai-organization/tensorrt_torchscript_demo/notebook_regseg/trt/regseg_withargmax.onnx"

In [3]:
def preprocess_image(img_path):
    # transformations for the input data
    transforms = Compose([
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # read input image
    input_img = cv2.imread(img_path)
    # do transformations
    input_data = transforms(input_img)
    batch_data = torch.unsqueeze(input_data, 0)
    return batch_data


def postprocess(output_data):
    # get class names
    with open("../data/imagenet_classes.txt") as f:
        classes = [line.strip() for line in f.readlines()]
    # calculate human-readable value by softmax
    confidences = torch.nn.functional.softmax(output_data, dim=1)[0] * 100
    # find top predicted classes
    _, indices = torch.sort(output_data, descending=True)
    i = 0
    # print the top classes predicted by the model
    while confidences[indices[0][i]] > 0.5:
        class_idx = indices[0][i]
        print(
            "class:",
            classes[class_idx],
            ", confidence:",
            confidences[class_idx].item(),
            "%, index:",
            class_idx.item(),
        )
        i += 1


In [4]:
wts = torch.load(WEIGHTS_PATH)

In [5]:
wts

{'model': OrderedDict([('stem.conv.weight',
               tensor([[[[-4.6607e-02, -2.2778e-01, -2.6104e-01],
                         [-1.7226e-02, -6.2845e-02, -2.9488e-02],
                         [-1.1728e-01, -2.0819e-01, -2.0240e-01]],
               
                        [[-1.9951e-01, -6.6667e-02,  3.0367e-02],
                         [-4.3388e-02, -2.2498e-02, -2.2349e-01],
                         [-9.9925e-02,  2.7143e-02, -1.0733e-01]],
               
                        [[-2.2153e-02,  2.3905e-02, -6.4726e-02],
                         [-1.6495e-01, -3.3830e-02, -8.2372e-02],
                         [-1.4322e-01, -1.0744e-01,  1.2894e-01]]],
               
               
                       [[[ 2.6681e-01,  4.5747e-01,  5.1579e-01],
                         [ 5.2484e-01,  2.9371e-01,  4.9036e-01],
                         [ 6.2717e-01,  2.9886e-01,  3.1188e-01]],
               
                        [[ 2.6005e-01,  3.0161e-01,  3.5218e-01],
             

###  Step 1 : Just Torch based inference
Here, we use a pretrained Resnet50 model to classify an input image. The inference in done purely in Pytorch. The model's prediction is passed through a post processing engine to get final predction.

In [5]:
input = preprocess_image("/home/zeeshan-motorai/workspace/POC/perception_deployment_manager/data/tegel_sample/115.jpg").cuda()
model = RegSeg(num_classes=2)
model.load_state_dict(wts['model'])
model.eval()
model.cuda()
# output = model(input)

# postprocess(output)


RegSeg(
  (stem): ConvBnAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU(inplace=True)
  )
  (body): RegSegBody(
    (stage4): DBlock(
      (gct): GCT()
      (conv1): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
      (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
      (conv3): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act3): ReLU(inplace=True)
      (shortcut): Shortcut(
        (avg): AvgPool2d

In [6]:
# import numpy as np
# output = output.cpu().detach().numpy()
# output = output.squeeze()
# normalizedData = (output-np.min(output))/(np.max(output)-np.min(output))
# # normalizedData
# cv2.imshow("", normalizedData)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [6]:
output[0]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0')

In [16]:
print(model)


RegSeg(
  (stem): ConvBnAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU(inplace=True)
  )
  (body): RegSegBody(
    (stage4): DBlock(
      (gct): GCT()
      (conv1): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
      (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
      (conv3): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act3): ReLU(inplace=True)
      (shortcut): Shortcut(
        (avg): AvgPool2d

In [6]:
from torchsummary import summary
summary(model, (3, 1200, 1920))

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 5.80 GiB total capacity; 3.75 GiB already allocated; 41.00 MiB free; 3.86 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### Step 2 : Convert Model to ONNX
Here, we first convert the given model to ONNX representation which will be later converted to TensorRT engine. There are several ways tto convert a model to TensorRT, but the most common method is using ONNX representation. We pass dummy input of the same shape as expected inputs, the model instance to the export function to convert a given torch network to ONNX.

In [6]:
# ONNX_FILE_PATH = '../onnx_files/regseg.onnx'
torch.onnx.export(model, input, ONNX_FILE_PATH, input_names=['input'],
                  output_names=['output'], export_params=True, verbose=True)

/home/zeeshan-motorai/anaconda3/envs/tensorrt8_6/lib/python3.10/site-packages/torch/onnx/_internal/jit_utils.py:258: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at ../torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1884.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/home/zeeshan-motorai/anaconda3/envs/tensorrt8_6/lib/python3.10/site-packages/torch/onnx/utils.py:687: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at ../torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1884.)
  _C._jit_pass_onnx_graph_shape_type_inference(


In [7]:
onnx.checker.check_model(onnx.load(ONNX_FILE_PATH))

### Step 3: Convert ONNX model to TensorRT engine
Now, we finally convert this generated ONNX model to TensorRT engine. This process involves several steps and we will look at them below :

The generated ONNX file is saved in the [onnx_files](../onnx_files) folder.

In [1]:
import pycuda.driver as cuda
import numpy as np
import tensorrt as trt
import pycuda.autoinit

#### 1. Create Builder
To create a builder, you must first create a logger. Then use the logger to create the builder.  Builder allows the creation of an optimized engine from a network definition. It allows the application to specify the maximum batch and workspace size, the minimum acceptable level of precision, timing iteration counts for autotuning, and an interface for quantizing networks to run in 8-bit precision. 

In [ ]:
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
builder = trt.Builder(TRT_LOGGER)

#### 2. Create Network
After the builder has been created, the first step in optimizing a model is to create a network definition. The EXPLICIT_BATCH flag is required in order to import models using the ONNX parser.  Network Definition provides methods for the application to specify the definition of a network. Input and output tensors can be specified, layers can be added, and there is an interface for configuring each supported layer type.
Layers like convolutional and recurrent layers, and a Plugin layer type allows the application to implement functionality not natively supported by TensorRT.

In [10]:
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

#### 3. Import model using ONNX Parser
Now, the network definition must be populated from the ONNX representation. You can create an ONNX parser to populate the network as follows:


In [11]:
parser = trt.OnnxParser(network, TRT_LOGGER)
success = parser.parse_from_file(ONNX_FILE_PATH)
for idx in range(parser.num_errors):
    print(parser.get_error(idx))

[05/11/2023-19:27:49] [TRT] [W] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.


#### 4. Building an engine
The next step is to create a build configuration specifying how TensorRT should optimize the model. This interface has many properties that you can set in order to control how TensorRT optimizes the network. 
 Allows the application to execute inference. 
   - It supports synchronous and asynchronous execution, profiling, and enumeration and querying of the bindings for the engine inputs and outputs. 
   - A single-engine can have multiple execution contexts, allowing a single set of trained parameters to be used for the simultaneous execution of multiple batches.

One important property is the maximum workspace size. Layer implementations often require a temporary workspace, and this parameter limits the maximum size that any layer in the network can use. If insufficient workspace is provided, it is possible that TensorRT will not be able to find an implementation for a layer:

In [ ]:
config = builder.create_builder_config()
# config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 22) # 1 MiB

After the configuration has been specified, the engine can be built and serialized with:

In [ ]:
serialized_engine = builder.build_serialized_network(network, config)

It may be useful to save the engine to a file for future use. You can do that like so:

In [ ]:
with open(ENGINE_PATH, "wb") as f:
    f.write(serialized_engine)

#### 5. Deserialize an Engine
To perform inference, deserialize the engine using the Runtime interface. Like the builder, the runtime requires an instance of the logger.

In [ ]:
runtime = trt.Runtime(TRT_LOGGER)

First load the engine from a file. Then deserialize the engine from a memory buffer:

In [ ]:
with open(ENGINE_PATH, "rb") as f:
    serialized_engine = f.read()

In [ ]:
engine = runtime.deserialize_cuda_engine(serialized_engine)

In [ ]:
engine.get_binding_shape(1)

In [ ]:
engine

#### 6. Performing Inference

The engine holds the optimized model, but to perform inference requires additional state for intermediate activations. An engine can have multiple execution contexts, allowing one set of weights to be used for multiple overlapping inference tasks. 

In [ ]:
context = engine.create_execution_context()

In [ ]:
engine.get_binding_shape(0)

Allocate some host and device buffers for inputs and outputs:

In [ ]:
# Determine dimensions and create page-locked memory buffers (i.e. won't be swapped to disk) to hold host inputs/outputs.
h_input = cuda.pagelocked_empty(trt.volume(engine.get_binding_shape(0)), dtype=np.float32)
h_output = cuda.pagelocked_empty(trt.volume(engine.get_binding_shape(1)), dtype=np.float32)
# Allocate device memory for inputs and outputs.
d_input = cuda.mem_alloc(h_input.nbytes)
d_output = cuda.mem_alloc(h_output.nbytes)
# Create a stream in which to copy inputs/outputs and run inference.
stream = cuda.Stream()

In [ ]:
host_input = np.array(preprocess_image("../data/regseg_sample.png").numpy(), dtype=np.float32, order='C')
# Transfer input data to the GPU.
cuda.memcpy_htod_async(d_input, host_input, stream)
# Run inference.
context. execute_async_v2(bindings=[int(d_input), int(d_output)], stream_handle=stream.handle)
# Transfer predictions back from the GPU.
cuda.memcpy_dtoh_async(h_output, d_output, stream)
# Synchronize the stream
stream.synchronize()

Create some space to store intermediate activation values. Since the engine holds the network definition and trained parameters, additional space is necessary. 

In [ ]:
output_data = torch.Tensor(h_output).unsqueeze(0)

In [ ]:
output_data

In [ ]:
output_data = output_data.reshape((1, 2, 1200, 1920))

Finally we are able to recreate the same results that we obtained using pure pytorch model. In conclusion, we first converted a given model to it's ONNX representation, then used this ONNX representation to generate a TensorRT engine. Then, use this saved engine for inference.

In [ ]:
output_data = output_data.cpu().numpy()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(output_data[0][1])